In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

GROQ_API_KEY= os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")
SERPER_API_KEY=os.getenv("SERPER_API_KEY")
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
NEO4J_URI="neo4j+s://4a1ee668.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="rHf5fPd0fExs1hJM5cSexQF3Qo7rLNsP0rm3daQYh1o"
AURA_INSTANCEID="4a1ee668"
AURA_INSTANCENAME="Free instance"


os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY 
os.environ["SERPER_API_KEY"] = SERPER_API_KEY
os.environ["LANGCHAIN_API_KEY"] =  LANGCHAIN_API_KEY
os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT
os.environ["GROQ_API_KEY"] = "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"
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD


In [2]:
from langchain_community.graphs import Neo4jGraph

In [3]:
graph=Neo4jGraph(
  url=NEO4J_URI,
  username=NEO4J_USERNAME,
  password=NEO4J_PASSWORD
)

C:\Users\thede\AppData\Local\Temp\ipykernel_22484\1759000609.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph=Neo4jGraph(


In [4]:
graph

In [5]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="deepseek-r1-distill-llama-70b")

In [6]:
llm.invoke("what is 6+2")

AIMessage(content='<think>\nFirst, I recognize that the user is asking for the sum of 6 and 2.\n\nTo solve this, I need to add the two numbers together.\n\nAdding 6 and 2 results in 8.\n\nTherefore, the answer to 6 plus 2 is 8.\n</think>\n\n**Solution:**\n\nTo find the sum of 6 and 2, follow these simple steps:\n\n1. **Start with the first number:**\n   \\[\n   6\n   \\]\n\n2. **Add the second number to the first number:**\n   \\[\n   6 + 2\n   \\]\n\n3. **Calculate the total:**\n   \\[\n   6 + 2 = 8\n   \\]\n\n**Final Answer:**\n\\[\n\\boxed{8}\n\\]', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 166, 'prompt_tokens': 9, 'total_tokens': 175, 'completion_time': 0.603636364, 'prompt_time': 0.000334845, 'queue_time': 0.061656794, 'total_time': 0.603971209}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_1bbe7845ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-2292b37c-3c91-4ec4-a97d-9697b76b533d-0', usage_metadata={'input_

In [7]:
from langchain_core.documents import Document

text = """
At 6 45 pm, on March 13th 2025, i had two vada pavs at shree mithai
"""

In [8]:
documents = [Document(page_content=text)]
documents

[Document(metadata={}, page_content='\nAt 6 45 pm, on March 13th 2025, i had two vada pavs at shree mithai\n')]

In [9]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

llm_transformer = LLMGraphTransformer(llm=llm)

In [10]:
graph_documents=llm_transformer.convert_to_graph_documents(documents)

In [11]:
graph_documents

[GraphDocument(nodes=[Node(id='I', type='Person', properties={}), Node(id='Vada Pav', type='Food', properties={}), Node(id='Shree Mithai', type='Place', properties={}), Node(id='6:45 Pm', type='Time', properties={}), Node(id='March 13Th, 2025', type='Date', properties={})], relationships=[Relationship(source=Node(id='I', type='Person', properties={}), target=Node(id='Vada Pav', type='Food', properties={}), type='HAD', properties={}), Relationship(source=Node(id='I', type='Person', properties={}), target=Node(id='6:45 Pm', type='Time', properties={}), type='AT', properties={}), Relationship(source=Node(id='I', type='Person', properties={}), target=Node(id='March 13Th, 2025', type='Date', properties={}), type='AT', properties={}), Relationship(source=Node(id='I', type='Person', properties={}), target=Node(id='Shree Mithai', type='Place', properties={}), type='AT', properties={})], source=Document(metadata={}, page_content='\nAt 6 45 pm, on March 13th 2025, i had two vada pavs at shree mi

In [12]:
graph.add_graph_documents(
 graph_documents,
 baseEntityLabel=True,
 include_source=True
)

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\thede\OneDrive\Desktop\langraph-end-to-end\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
from langchain_community.vectorstores import Neo4jVector
from typing import Tuple,List,Optional

In [ ]:
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=['text'],
    embedding_node_property="embedding",
)

In [16]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [17]:
from langchain_core.pydantic_v1 import BaseModel, Field

class Entities(BaseModel):
  """ Identifying information from """

  names: List[str] = Field(
    ...,
    description="all the person, organization or business entities that appear in the text"
  )

c:\Users\thede\OneDrive\Desktop\langraph-end-to-end\venv\Lib\site-packages\IPython\core\interactiveshell.py:3549: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [19]:
prompt = ChatPromptTemplate.from_messages(
  [
    (
      "system",
      "You are extracting relevant entities like organization, person, food etc from the text",
    ),
    ("system",
     "Use the given format to extract information from the following"
     "input: {question}"),
  ]
)

In [20]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [21]:
entity_chain.invoke({"question":"what did i have at shree mithai"})

Entities(names=['shree mithai'])

In [22]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [23]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f"{word}~2 AND" 
    full_text_query += f"{words[-1]}~2"
    return full_text_query.strip()


In [24]:
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """
            CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node, score
            CALL {
                WITH node
                MATCH (node)-[r:MENTIONS]->(neighbor)
                RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
                UNION ALL
                WITH node
                MATCH (node)<-[r:MENTIONS]-(neighbor)
                RETURN neighbor.id + ' - ' + type(r) + ' -> ' + node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result


In [25]:
print(structured_retriever("what did i eat at shree mithai"))

C:\Users\thede\AppData\Local\Temp\ipykernel_22484\4044109123.py:3: LangChainDeprecationWarning: The function `remove_lucene_chars` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the function exists in the :meth:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :meth:`~langchain-neo4j` and import as `from :meth:`~langchain_neo4j.vectorstores.neo4j_vector import remove_lucene_chars``.
  words = [el for el in remove_lucene_chars(input).split() if el]
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 4, column: 13, offset: 118} for query: "\n            CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n     

37e97ecf516d1dd7eec6d70d1a822ee1 - MENTIONS -> Shree Mithai


In [26]:
def retriever(question : str):
  print(f"Search query: {question}")
  structured_data = structured_retriever(question)
  unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
  final_data = f"""
    Structured Data:
    {structured_data}
Unstructured data:
{"Document ".join(unstructured_data)}
"""
  return final_data

In [28]:
_template = """
Given the following conversation and a follow up question, rephrase the follow up question
to be a standalone question, in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:
"""

In [29]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [30]:
from langchain_core.messages import HumanMessage,SystemMessage,AIMessage

In [31]:
def format_chat_history(chat_history : List[Tuple[str,str]]) -> List:
  buffer = []
  for human, ai in chat_history:
    buffer.append(HumanMessage(content=human))
    buffer.append(AIMessage(content=ai))
  return buffer

In [43]:
from langchain_core.runnables import RunnableBranch,RunnableLambda,RunnablePassthrough,RunnableParallel

In [38]:
from langchain_core.output_parsers import StrOutputParser

In [40]:
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    
    (
      RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
        run_name="HasChatHistoryCheck"
    ),
        # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: format_chat_history(x["chat_history"])
        ) |
        CONDENSE_QUESTION_PROMPT |
        ChatGroq(
          model="deepseek-r1-distill-llama-70b",
          temperature=0
        ) |
        StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x: x["question"]),
)


In [41]:
template = """ Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.

Answer:

"""

In [42]:
prompt = ChatPromptTemplate.from_template(template)

In [44]:
chain = (
  RunnableParallel(
    {
      "context":_search_query | retriever,
      "question":RunnablePassthrough(),
    }
  )
  | prompt
  |llm
  | StrOutputParser()
)

In [ ]:
chain.invoke({"question":"How many vada pav did i eat"})

Search query: How many vada pav did i eat


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 4, column: 13, offset: 118} for query: "\n            CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node, score\n            CALL {\n                WITH node\n                MATCH (node)-[r:MENTIONS]->(neighbor)\n                RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n                UNION ALL\n                WITH node\n                MATCH (node)<-[r:MENTIONS]-(neighbor)\n                RETURN neighbor.id + ' - ' + type(r) + ' -> ' + node.id AS output\n            }\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARN

'<think>\nOkay, so I need to figure out how many vada pav the person ate. Let me look at the information provided. \n\nIn the structured data, there\'s a mention of "Vada Pav," which is the dish in question. Then, in the unstructured text, it says, "At 6 45 pm, on March 13th 2025, i had two vada pavs at shree mithai." \n\nSo, the key part here is "had two vada pavs." That clearly states the number. I don\'t think there\'s any ambiguity here. The person explicitly mentions having two of them. \n\nI should make sure I\'m only using the provided context and not adding any outside information. The question is straightforward, asking for the count, so the answer should be concise and based solely on the given text.\n\nI don\'t see any other numbers or mentions of vada pav elsewhere, so two is the correct answer. No need to overcomplicate it.\n</think>\n\nYou ate two vada pavs.'

: 

OCR

In [2]:
import easyocr as eocr
reader = eocr.Reader(['en'])
result = reader.readtext("istockphoto-1758363728-612x612.jpg")

for (bbox,text,prob) in result:
  print(text)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


c:\Users\thede\OneDrive\Desktop\langraph-end-to-end\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


The
best
preparation
for
tomorrow
is
doing your
best today -
